# Парсер статей ГК РФ

In [94]:
from bs4 import BeautifulSoup
import pathlib, re, pandas as pd

In [95]:
def read_file_text(path: pathlib.Path) -> str:
    """Reads a .doc file saved as HTML and returns plain text"""
    data = path.read_bytes()

    if b'<html' in data.lower():
        soup = BeautifulSoup(data, 'html.parser')
        text = soup.get_text('\n')
    else:
        try:
            text = data.decode('utf-8')
        except UnicodeDecodeError:
            text = data.decode('cp1251', errors='ignore')
    return text.replace('\u00A0', ' ')

In [96]:
# Regular Expression Patterns
re_section = re.compile(r'^РАЗДЕЛ\s+([IVXLCDM]+)\.?\s*(.*)$', re.I | re.M)
re_chapter = re.compile(r'^Глава\s+(\d+)\.?\s*(.*)$', re.I | re.M)
re_article = re.compile(r'^Статья\s+(\d+)\.?\s*(.*)$', re.I | re.M)

In [97]:
# Roman Numeral Converter
def roman_to_int(s: str) -> int:
    vals = dict(I=1, V=5, X=10, L=50, C=100, D=500, M=1000)
    total = 0
    prev = 0
    for ch in reversed(s.upper()):
        v = vals.get(ch, 0)
        total += -v if v < prev else v
        prev = v
        
    return total or None

In [98]:
def parse_articles(path: pathlib.Path):

    def _save_article():
        """Saves the accumulated article in articles_meta"""
        article_end_char = sum(len(l) for l in lines[:i])
        article_text = text[article_start_char:article_end_char]
        articles_meta.append({
            'file': path.name,
            'article_num': article_num,
            'article_title': article_title,
            'article_text': article_text,
            'section_title': current_section_title,
            'chapter_title': current_chapter_title,
            'start_char': article_start_char,
            'end_char': article_end_char,
            'text_length': len(article_text)
        })

    def _peek_for_title(lines, start_idx):
        """
        Returns the first "meaningful" line after 'Article N'.
        Ignores empty, '1', '1.', '.', '2.', etc.
        """
        for ln in lines[start_idx:]:
            candidate = ln.strip()
            if candidate == '':
                continue
            if re.match(r'^[\d.]+$', candidate):
                continue
            # trim possible '1.' at the beginning of the line (our bugfix)
            candidate = re.sub(r'^\d+\s*\.?\s*', '', candidate).strip()
            if candidate:
                return candidate
        return ''


    text = read_file_text(path)
    lines = text.splitlines(keepends=True)

    articles_meta = []
    current_section_title = ''
    current_chapter_title = ''
    article_num = None
    article_title = ''
    article_start_char = None

    for i, raw_line in enumerate(lines):
        line = raw_line.strip()

        # --- раздел -------------------------------------
        m_sec = re_section.match(line)
        if m_sec:
            # save previous article if there was one
            if article_num is not None:
                _save_article()
            current_section_title = m_sec.group(2).strip()

        # --- глава --------------------------------------
        m_ch = re_chapter.match(line)
        if m_ch:
            if article_num is not None:
                _save_article()
            current_chapter_title = m_ch.group(2).strip()

        # --- статья -------------------------------------
        m_art = re_article.match(line)
        if m_art:
            # close the previous one
            if article_num is not None:
                _save_article()

            article_num = int(m_art.group(1))
            article_title = m_art.group(2).strip()

            # bugfix: if the title is empty, we search further
            if article_title == '':
                article_title = _peek_for_title(lines, i + 1)

            article_start_char = sum(len(l) for l in lines[:i])

    if article_num is not None:
        _save_article()

    return articles_meta

In [99]:
data_dir = pathlib.Path('/Users/theother_archee/CursorProjects/SmartClause/parser/dataset')
files = sorted(data_dir.glob('Part*.doc'))  # Search Part1.doc, Part2.doc, Part3.doc, Part4.doc

In [100]:
files

[PosixPath('/Users/theother_archee/CursorProjects/SmartClause/parser/dataset/Part1.doc'),
 PosixPath('/Users/theother_archee/CursorProjects/SmartClause/parser/dataset/Part2.doc'),
 PosixPath('/Users/theother_archee/CursorProjects/SmartClause/parser/dataset/Part3.doc'),
 PosixPath('/Users/theother_archee/CursorProjects/SmartClause/parser/dataset/Part4.doc')]

In [101]:
all_articles = []
for f in files:
    all_articles.extend(parse_articles(f))

df = pd.DataFrame(all_articles)
print(df.head())

        file  article_num                                      article_title  \
0  Part1.doc            1      Основные начала гражданского законодательства   
1  Part1.doc            2  Отношения, регулируемые гражданским законодате...   
2  Part1.doc            3  Гражданское законодательство и иные акты, соде...   
3  Part1.doc            4  Действие гражданского законодательства во времени   
4  Part1.doc            5                                             Обычаи   

                                        article_text    section_title  \
0  Статья 1. Основные начала гражданского законод...  Общие положения   
1  Статья 2. Отношения, регулируемые гражданским ...  Общие положения   
2  Статья 3. Гражданское законодательство и иные ...  Общие положения   
3  Статья 4. Действие гражданского законодательст...  Общие положения   
4  Статья 5. Обычаи \n(Наименование в редакции Фе...  Общие положения   

                  chapter_title  start_char  end_char  text_length  
0  Гражданс

## Check if data was parsed correctly

In [102]:
print("Shape: ", df.shape)
print("Columns: ", df.columns)

Shape:  (1815, 9)
Columns:  Index(['file', 'article_num', 'article_title', 'article_text', 'section_title',
       'chapter_title', 'start_char', 'end_char', 'text_length'],
      dtype='object')


In [103]:
for col in df.columns:
    print(f"Уникальные значения в столбце '{col}':")
    print(df[col].unique())
    print("-" * 40)


Уникальные значения в столбце 'file':
['Part1.doc' 'Part2.doc' 'Part3.doc' 'Part4.doc']
----------------------------------------
Уникальные значения в столбце 'article_num':
[   1    2    3 ... 1539 1540 1541]
----------------------------------------
Уникальные значения в столбце 'article_title':
['Основные начала гражданского законодательства'
 'Отношения, регулируемые гражданским законодательством'
 'Гражданское законодательство и иные акты, содержащие нормы гражданского права'
 ... 'Исключительное право на коммерческое обозначение'
 'Действие исключительного права на коммерческое обозначение'
 'Соотношение права на коммерческое обозначение с правами на фирменное наименование и товарный знак']
----------------------------------------
Уникальные значения в столбце 'article_text':
['Статья 1. Основные начала гражданского законодательства\n \n1. Гражданское законодательство основывается на признании равенства участников регулируемых им отношений, неприкосновенности собственности, свобод

In [104]:
# Checking for empty or NaN lines
print(df.isnull().sum())

for col in df.columns:
    num_empty = (df[col].astype(str).str.strip() == '').sum()
    print(f"Пустых строк в '{col}': {num_empty}")


file             0
article_num      0
article_title    0
article_text     0
section_title    0
chapter_title    0
start_char       0
end_char         0
text_length      0
dtype: int64
Пустых строк в 'file': 0
Пустых строк в 'article_num': 0
Пустых строк в 'article_title': 0
Пустых строк в 'article_text': 0
Пустых строк в 'section_title': 341
Пустых строк в 'chapter_title': 28
Пустых строк в 'start_char': 0
Пустых строк в 'end_char': 0
Пустых строк в 'text_length': 0


In [105]:
empty_titles = df[df['article_title'].astype(str).str.strip() == '']
print(empty_titles[['article_title', 'article_text', 'chapter_title']].iloc[:20])

Empty DataFrame
Columns: [article_title, article_text, chapter_title]
Index: []
